# Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Concatenate, Rescaling
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

from tensorflow.keras import backend as K
import gc

# Parameters

In [2]:
# NN params
epochs = 25
batch_size = 16

# Loading data

In [3]:
df = pd.read_csv("houses_preprocessed.csv")

In [4]:
df.head()

,n_citi,bed,bath,sqft,price,image
0,-1.500387,-0.489366,-0.472771,-1.424855,228500,houses_preprocessed/1.jpg
1,-0.574868,-0.489366,-1.515838,-1.340002,273950,houses_preprocessed/2.jpg
2,-1.500387,-0.489366,-1.515838,-1.064963,350000,houses_preprocessed/3.jpg
3,-1.438092,0.477001,0.570296,0.363878,385100,houses_preprocessed/4.jpg
4,-1.500387,-0.489366,-1.515838,-1.064963,350000,houses_preprocessed/5.jpg


# Experimental set up

In [5]:
# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(df.drop(['price'], axis=1), df['price'], test_size=0.2, random_state=42)

# Now, separate tabular and image data for each split
X_train_tab = X_train.drop(['image'], axis=1).values # np
X_train_img = X_train['image'] # pd 

X_test_tab = X_test.drop(['image'], axis=1).values # np
X_test_img = X_test['image'] # pd 

# Print shapes
print("Training Data Shapes:")
print(f"Tabular features: {X_train_tab.shape}")
print(f"Image features: {X_train_img.shape}")
print(f"Target prices: {y_train.shape}")
print("\nTest Data Shapes:")
print(f"Tabular features: {X_test_tab.shape}")
print(f"Image features: {X_test_img.shape}")
print(f"Target prices: {y_test.shape}")

Training Data Shapes:
Tabular features: (12237, 4)
Image features: (12237,)
Target prices: (12237,)

Test Data Shapes:
Tabular features: (3060, 4)
Image features: (3060,)
Target prices: (3060,)


# Modeling and Performance metrics

## Neural Networks

In [6]:
def base_nn(input_size_tabular):
    # Image processing branch
    img_input = Input(shape=(311, 415, 3), name='image_input')
    x = Conv2D(32, (3, 3), activation='relu')(img_input)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    
    # Tabular data processing branch
    tabular_input = Input(shape=(input_size_tabular,), name='tabular_input')
    y = Dense(64, activation='relu')(tabular_input)
    y = Dense(32, activation='relu')(y)
    
    # Combine both branches
    combined = Concatenate()([x, y])
    z = Dense(64, activation='relu')(combined)
    output = Dense(1)(z) # Regression output for price prediction
    
    nn_model = Model(inputs=[img_input, tabular_input], outputs=output)
    
    # Compile the model
    nn_model.compile(optimizer='adam',
                  loss='mae',
                  metrics=['mae', 'R2Score'])
    
    # Display model summary debug
    # nn_model.summary()

    return nn_model

In [7]:
def resnet_nn(input_size_tabular):
    # Image processing branch with pre-trained ResNet50
    res_net = ResNet50(weights='imagenet', include_top=False, input_shape=(311, 415, 3))
    
    # Unfreeze only the last 10 layers of resnet (fine-tuning) 
    res_net.trainable = False 
    for layer in res_net.layers[-10:]:
        layer.trainable = True

    # Image processing branch
    img_input = Input(shape=(311, 415, 3), name='image_input')
    x = res_net(img_input)
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    
    # Tabular data processing branch
    tabular_input = Input(shape=(input_size_tabular,), name='tabular_input')
    y = Dense(64, activation='relu')(tabular_input)
    y = Dense(32, activation='relu')(y)
    
    # Combine both branches
    combined = Concatenate()([x, y])
    z = Dense(64, activation='relu')(combined)
    output = Dense(1)(z)  # Regression output for price prediction
    
    # Define the model
    res_net_model = Model(inputs=[img_input, tabular_input], outputs=output)
    
    # Compile the model
    res_net_model.compile(optimizer='adam', 
                          loss='mae',
                          metrics=['mae', 'R2Score'])
   
    # Display model summary debug
    # res_net_model.summary()

    return res_net_model

In [8]:
'''
I did not write this code, the code is from: https://www.tensorflow.org/tutorials/load_data/images
It helps us train the NN more dynamically, it loads images on the go, such that not all RAM is used up.
It does try to maximise RAM usage this is basically what the tf.data.AUTOTUNE does.
'''

# Loads an image and normalizes it from [0,1]
def process_example(image_path, tabular_features, label):
    # Load raw bytes and convert to RGB
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)

    # Normalize image to [0, 1] and convert to float32
    image = tf.image.convert_image_dtype(image, tf.float32)

    return (image, tabular_features), label


# Creates on the fly data sets to train/test the model, we need this to not exceed memory
def create_dataset(image_paths, tabular_data, labels, shuffle=True):
    # Convert to tensors
    image_paths = tf.convert_to_tensor(image_paths)
    tabular_data = tf.convert_to_tensor(tabular_data, dtype=tf.float32)
    labels = tf.convert_to_tensor(labels, dtype=tf.float32)

    # Build dataset
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, tabular_data, labels))
    dataset = dataset.map(lambda img, tab, lbl: process_example(img, tab, lbl), num_parallel_calls=tf.data.AUTOTUNE)
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return dataset


def train_and_evaluate_nn(nn, 
                          X_train_img_paths, X_train_tab, y_train,
                          X_test_img_paths, X_test_tab, y_test,
                          verbose=1):

    # Dynamic dataset loading
    train_ds = create_dataset(X_train_img_paths, X_train_tab, y_train, shuffle=True) # Shuffle to break ordering
    test_ds = create_dataset(X_test_img_paths, X_test_tab, y_test, shuffle=False) # No shuffle, we arent learning, just predicting

    # Train and Test
    history = nn.fit(train_ds, epochs=epochs, verbose=verbose)
    test_loss, test_mae, r2 = nn.evaluate(test_ds, verbose=0)

    return history, test_loss, test_mae, r2

## Logistic Regression
On just the tabular features

In [9]:
def train_and_evaluate_lin_model(model, X_train_tab, y_train, X_test_tab, y_test):
    # Train the model
    model.fit(X_train_tab, y_train)
    
    # Evaluate the model
    y_test_pred = model.predict(X_test_tab)
    mae_test = mean_absolute_error(y_test, y_test_pred)
    r2 = r2_score(y_test, y_test_pred)
    
    return mae_test, r2

### Train and Evaluate

In [10]:
# Create NNs with tabular features = 4 (n_citi, bed, bath, sqft)
nn_base = base_nn(4)
nn_resnet = resnet_nn(4)
lin = LinearRegression()

In [11]:
# NN
print("Training Base NN")
nn_base_hist, _, nn_base_mae, nn_base_r2 = train_and_evaluate_nn(nn_base, X_train_img, X_train_tab, y_train, X_test_img, X_test_tab, y_test)
print(f"NN Base MAE: {nn_base_mae:.0f}\nNN Base R2: {nn_base_r2:.2f}")

# Try to clear NN from memory
K.clear_session()
gc.collect()

Training Base NN
Epoch 1/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 446s 554ms/step - R2Score: -1.1422 - loss: 405263.1875 - mae: 405263.1875
Epoch 2/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 391s 504ms/step - R2Score: 0.0412 - loss: 262758.2500 - mae: 262758.2500
Epoch 3/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 414s 534ms/step - R2Score: 0.2272 - loss: 235915.6250 - mae: 235915.6250
Epoch 4/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 404s 522ms/step - R2Score: 0.2701 - loss: 228518.2812 - mae: 228518.2812
Epoch 5/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 391s 505ms/step - R2Score: 0.3022 - loss: 224010.7656 - mae: 224010.7656
Epoch 6/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 418s 540ms/step - R2Score: 0.3215 - loss: 222920.3750 - mae: 222920.3750
Epoch 7/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 418s 539ms/step - R2Score: 0.3324 - loss: 220288.2656 - mae: 220288.2656
Epoch 8/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 399s 515ms/step - R2Score: 0.3310 - loss: 220313.8750 - mae: 220313.8750
Epoch 9/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 393s 506ms/step - R2Score: 0.3301 - lo

0

In [12]:
# Resnet
print("Training Resnet")
nn_resnet_hist, _, nn_resnet_mae, nn_resnet_r2 = train_and_evaluate_nn(nn_resnet, X_train_img, X_train_tab, y_train, X_test_img, X_test_tab, y_test)
print(f"Resnet MAE: {nn_resnet_mae:.0f}\nResnet R2: {nn_resnet_r2:.2f}")

# Try to clear NN from memory
K.clear_session()
gc.collect()

Training Resnet
Epoch 1/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 961s 1s/step - R2Score: -2.4521 - loss: 563919.8750 - mae: 563919.8750
Epoch 2/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 955s 1s/step - R2Score: 0.1991 - loss: 239459.4531 - mae: 239459.4531
Epoch 3/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 940s 1s/step - R2Score: 0.3014 - loss: 224556.1562 - mae: 224556.1562
Epoch 4/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 955s 1s/step - R2Score: 0.3154 - loss: 219261.7188 - mae: 219261.7188
Epoch 5/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 956s 1s/step - R2Score: 0.3266 - loss: 217687.0781 - mae: 217687.0781
Epoch 6/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 964s 1s/step - R2Score: 0.3433 - loss: 216634.7188 - mae: 216634.7188
Epoch 7/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 954s 1s/step - R2Score: 0.3601 - loss: 214348.9688 - mae: 214348.9688
Epoch 8/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 955s 1s/step - R2Score: 0.3454 - loss: 211343.7031 - mae: 211343.7031
Epoch 9/25
765/765 ━━━━━━━━━━━━━━━━━━━━ 953s 1s/step - R2Score: 0.3714 - loss: 210201.9062 - mae: 21020

0

In [13]:
# LR
print("Training LR")
lr_mae, lr_r2 = train_and_evaluate_lin_model(lin, X_train_tab, y_train, X_test_tab, y_test)
print(f"LR MAE: {lr_mae:.0f}\nLR R2: {lr_r2:.2f}")

Training LR
LR MAE: 223187
LR R2: 0.35
